In [41]:
# import sys
# from  ift6758.data.data_acquisition import Season
# season2018 = Season(2018,"../ift6758/data")
# df_season2018_clean = season2018.clean_data()
# df[["gamePk","liveData.plays.allPlays"]]

In [8]:
from  ift6758.data.data_acquisition import Season

from ipywidgets import widgets, interact, RadioButtons, IntSlider, Output, Layout, Dropdown
from ipywidgets.embed import embed_minimal_html

from scipy.ndimage import gaussian_filter

import plotly.graph_objects as go

import json
import pandas as pd
import numpy as np
from tabulate import tabulate
import matplotlib.pyplot as plt
from matplotlib import cm
import seaborn as sns

import requests
import json

In [9]:
def time_played(row):
    """
    return time in mins
    """
    # print(type(row['about.periodTime']))
    if row['result.event'] == 'GOAL' and row['about.period']>3:
        # return row['about.periodTime'] + row['about.periodTime']
        # .date.min()
        min_sec1 = row['about.periodTime'].split(':')
        return float(min_sec1[0]) + float(min_sec1[1])/60
    else:
        # return row['about.periodTime']
        # .date.min()
        min_sec1 = row['about.periodTime'].split(':')
        min_sec2 = row['about.periodTimeRemaining'].split(':')
        return float(min_sec1[0]) + float(min_sec2[0]) + (float(min_sec1[1]) +  float(min_sec2[1]))/60


In [11]:
from ift6758.visualizations.advanced_visualisation import advanced_visualization

File already Exists, loading from ../ift6758/data/PICKLE//2016_clean.pkl
num_teams 30 df_clean shape (80399, 14)
df_clean shape after concatenating coordinates (80399, 16)
games played by teams {'Anaheim Ducks': 99, 'Arizona Coyotes': 82, 'Boston Bruins': 88, 'Buffalo Sabres': 82, 'Calgary Flames': 86, 'Carolina Hurricanes': 82, 'Chicago Blackhawks': 86, 'Colorado Avalanche': 82, 'Columbus Blue Jackets': 87, 'Dallas Stars': 82, 'Detroit Red Wings': 82, 'Edmonton Oilers': 95, 'Florida Panthers': 82, 'Los Angeles Kings': 82, 'Minnesota Wild': 87, 'Montréal Canadiens': 88, 'Nashville Predators': 104, 'New Jersey Devils': 82, 'New York Islanders': 82, 'New York Rangers': 94, 'Ottawa Senators': 101, 'Philadelphia Flyers': 82, 'Pittsburgh Penguins': 107, 'San Jose Sharks': 88, 'St. Louis Blues': 93, 'Tampa Bay Lightning': 82, 'Toronto Maple Leafs': 88, 'Vancouver Canucks': 82, 'Washington Capitals': 95, 'Winnipeg Jets': 82}
Empty DataFrame
Columns: [team.name, x_bin, y_bin, shots_average]
In

In [13]:
IMG_PATH = "../figures/half_nhl_rink_h.png"

In [14]:
url = 'https://statsapi.web.nhl.com/api/v1/teams'

r = requests.get(url, headers={'accept': 'application/json'})
json_teams = r.json()

# print(json_teams["teams"])

In [15]:
team_list = list()
for team in json_teams["teams"]:
    team_list.append(team["name"])
print(team_list)

['New Jersey Devils', 'New York Islanders', 'New York Rangers', 'Philadelphia Flyers', 'Pittsburgh Penguins', 'Boston Bruins', 'Buffalo Sabres', 'Montréal Canadiens', 'Ottawa Senators', 'Toronto Maple Leafs', 'Carolina Hurricanes', 'Florida Panthers', 'Tampa Bay Lightning', 'Washington Capitals', 'Chicago Blackhawks', 'Detroit Red Wings', 'Nashville Predators', 'St. Louis Blues', 'Calgary Flames', 'Colorado Avalanche', 'Edmonton Oilers', 'Vancouver Canucks', 'Anaheim Ducks', 'Dallas Stars', 'Los Angeles Kings', 'San Jose Sharks', 'Columbus Blue Jackets', 'Minnesota Wild', 'Winnipeg Jets', 'Arizona Coyotes', 'Vegas Golden Knights', 'Seattle Kraken']


In [16]:
len(team_list)

32

In [18]:
#CHOOSE GAME ID
season_selector = IntSlider(
    value=2016,
    min=2016,
    max=2020,
    step=1,
    description='Season: ',
    continuous_update=False, #only reassign value when the slider stops moving
    layout=Layout(width='50%')
)

#CHOOSE GAME TYPE
team_selector = Dropdown(
    options=team_list,
    value='New Jersey Devils',
    description='Number:',
    disabled=False,
    continuous_update=False,
    layout=Layout(width='50%')
)

container = widgets.HBox(children=[season_selector, team_selector])

##

from random import randint
import plotly.express as px
import plotly.graph_objects as go
from skimage import io

#Z = np.random.uniform(-1,1,(85,100))
#trace1 = go.Histogram(x=Z, opacity=0.75, name='Departure Delays')

layout = go.Layout(title='Loading')

img = io.imread(IMG_PATH)
# figure = px.imshow(img,
#                    extent=[],
#                    #x=np.arange(-42.5,43.5),
#                    #y=np.arange(-100,101),
#                    aspect='auto',
#                   )

#     scaleanchor = "x",
#     scaleratio = 1
# )
f = advanced_visualization(year=2016,team_name="New Jersey Devils",bin_width=1)

figure = go.Figure(layout_xaxis_range=[0,100],
                  layout_yaxis_range=[0,85])

figure.update_xaxes(showgrid=False)
figure.update_yaxes(showgrid=False)

figure.update_layout(
    title="Choose a Team and/or a Season",
    xaxis_title="Distance from the Rink's Center Line (feet)",
    yaxis_title="Distance from the Rink's Bottom Edge (feet)",
    legend_title="Legend Title",
    font=dict(
        size=10,
        family='Arial, sans-serif'
    ),
    width=550,
    height=467,
)
#img size: 550 x 467


figure.add_trace (
    go.Contour(
        z=f,
        # cmapk= 'magma',
        colorscale='RdBu_r',
        line_smoothing=1,
        opacity=0.6,
        connectgaps=True,
        contours = dict(start=-1.1,end=1.1,size=0.2),
        ncontours=25,

        #colorbar setup
        colorbar=dict(
            title='Excess Shots', # title here
            titleside='right',
            titlefont=dict(
                size=10,
                family='Arial, sans-serif'),
            #TICKS SETUP OF COLORBAR
            nticks=10, ticks='outside',
            ticklen=1, tickwidth=1,
            showticklabels=True,
            tickangle=0, tickfont_size=12
        )
    ))

# Add images
figure.add_layout_image(
        dict(
            source=IMG_PATH,
            xref="x2 domain",
            yref="y2 domain",
            x=0,
            y=1,
            sizex=1,
            sizey=1,
            opacity=1,
            sizing = "stretch",
            layer="below")
)

#figure.update_layout(template="plotly_white")

g = go.FigureWidget(figure)


#LINK team to widget 
def update_fig(change):
    
    season = season_selector.value
    team = team_selector.value
    
    title = team + '\n' + "Season: " + str(season) + "  Relative to League Average for the Season "
    
    f = advanced_visualization(year=season,team_name=team,bin_width=1,sigma=2)

    with g.batch_update():
            #g.
            g.data[0].z = f
            #g.data[1].x = x2
            g.layout.title = title
            #g.layout.xaxis.title = 'Delay in Minutes'
            #g.layout.yaxis.title = 'Number of Delays'
            
season_selector.observe(update_fig, 'value')
team_selector.observe(update_fig, 'value')

widgets.VBox([container,g])

File already Exists, loading from ../ift6758/data/PICKLE//2016_clean.pkl
num_teams 30 df_clean shape (80399, 14)
df_clean shape after concatenating coordinates (80399, 16)
games played by teams {'Anaheim Ducks': 99, 'Arizona Coyotes': 82, 'Boston Bruins': 88, 'Buffalo Sabres': 82, 'Calgary Flames': 86, 'Carolina Hurricanes': 82, 'Chicago Blackhawks': 86, 'Colorado Avalanche': 82, 'Columbus Blue Jackets': 87, 'Dallas Stars': 82, 'Detroit Red Wings': 82, 'Edmonton Oilers': 95, 'Florida Panthers': 82, 'Los Angeles Kings': 82, 'Minnesota Wild': 87, 'Montréal Canadiens': 88, 'Nashville Predators': 104, 'New Jersey Devils': 82, 'New York Islanders': 82, 'New York Rangers': 94, 'Ottawa Senators': 101, 'Philadelphia Flyers': 82, 'Pittsburgh Penguins': 107, 'San Jose Sharks': 88, 'St. Louis Blues': 93, 'Tampa Bay Lightning': 82, 'Toronto Maple Leafs': 88, 'Vancouver Canucks': 82, 'Washington Capitals': 95, 'Winnipeg Jets': 82}
Empty DataFrame
Columns: [team.name, x_bin, y_bin, shots_average]
In

In [28]:
for year in [2016,2017,2018,2019,2020]:
    for team in team_list:
        f=advanced_visualization(year,team_name=team)
        figure = go.Figure(layout_xaxis_range=[0,100],
                        layout_yaxis_range=[0,85])
        figure.update_xaxes(showgrid=False)
        figure.update_yaxes(showgrid=False)

        figure.update_layout(
            title = f"{team}  Season: {year} Relative to League Average for the Season",
            xaxis_title="Distance from the Rink's Center Line (feet)",
            yaxis_title="Distance from the Rink's Bottom Edge (feet)",
            legend_title="Legend Title",
            font=dict(
                size=10,
                family='Arial, sans-serif'
            ),
            width=550,
            height=467,
        )
        #img size: 550 x 467


        figure.add_trace (
            go.Contour(
                z=f,
                # cmapk= 'magma',
                colorscale='RdBu_r',
                line_smoothing=1,
                opacity=0.6,
                connectgaps=True,
                contours = dict(start=-1.1,end=1.1,size=0.2),
                ncontours=25,

                #colorbar setup
                colorbar=dict(
                    title='Excess Shots', # title here
                    titleside='right',
                    titlefont=dict(
                        size=10,
                        family='Arial, sans-serif'),
                    #TICKS SETUP OF COLORBAR
                    nticks=10, ticks='outside',
                    ticklen=1, tickwidth=1,
                    showticklabels=True,
                    tickangle=0, tickfont_size=12
                )
            ))

        figure.add_layout_image(
                dict(
                    source=IMG_PATH,
                    xref="x2 domain",
                    yref="y2 domain",
                    x=0,
                    y=1,
                    sizex=1,
                    sizey=1,
                    opacity=1,
                    sizing = "stretch",
                    layer="below")
        )
        # figure.show()
        figure.write_html(f"../figures/{year}_{team}.html")
        break

    break

File already Exists, loading from ../ift6758/data/PICKLE//2016_clean.pkl
num_teams 30 df_clean shape (80399, 14)
df_clean shape after concatenating coordinates (80399, 16)
games played by teams {'Anaheim Ducks': 99, 'Arizona Coyotes': 82, 'Boston Bruins': 88, 'Buffalo Sabres': 82, 'Calgary Flames': 86, 'Carolina Hurricanes': 82, 'Chicago Blackhawks': 86, 'Colorado Avalanche': 82, 'Columbus Blue Jackets': 87, 'Dallas Stars': 82, 'Detroit Red Wings': 82, 'Edmonton Oilers': 95, 'Florida Panthers': 82, 'Los Angeles Kings': 82, 'Minnesota Wild': 87, 'Montréal Canadiens': 88, 'Nashville Predators': 104, 'New Jersey Devils': 82, 'New York Islanders': 82, 'New York Rangers': 94, 'Ottawa Senators': 101, 'Philadelphia Flyers': 82, 'Pittsburgh Penguins': 107, 'San Jose Sharks': 88, 'St. Louis Blues': 93, 'Tampa Bay Lightning': 82, 'Toronto Maple Leafs': 88, 'Vancouver Canucks': 82, 'Washington Capitals': 95, 'Winnipeg Jets': 82}
Empty DataFrame
Columns: [team.name, x_bin, y_bin, shots_average]
In